# Extract the Codebook HTML into a dataframe and inject into Mapping Excel

This notebook is capable of converting a local downloaded version of the ISARIC Codebook (save as single web page) into a dataframe containg the:
ID, Variable / Field Name, Field Label, Field Attributes, Dictionary (option value and label)

Usage:
- Save the ISARIC Codebook as Single web page in ./ISARIC Codebook
- Put the mapping_sheet in ./ISARIC Codebook
   (this prevents accidental overwerite)
- Run
- Legenda: compared to ISARIC Codebook
  - OBSOLETE: cells in Column A are empty and in Column C marked RED
  - NEW: cells in Column C are marked GREEN, if applicable Columns K and L are prefilled to make value mapping easier
  - CHANGED: changed cells are marked ORANGE

If you want to play with this Notebook, open the **ISARIC_EraCORE_mapping.xlsx**, delete the sheet **ISARIC_EraCORE_mapping** and copy **ISARIC_EraCORE_mapping_old** and rename this **ISARIC_EraCORE_mapping**

In [1]:
# import re
import pandas as pd
import openpyxl as xl
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from tabulate import tabulate
from datetime import datetime
# from bs4 import BeautifulSoup                       # Beautiful Soup Vers
import generic_functions2 as gf

In [2]:
# choose the ISARIC Codebook

# folder = './'
# print('\n'*2)
# gf.print_title('Select the Excel containing all the configuration & mapping')
# conversion_excel = gf.choose_dir_item(folder,'files', 'xlsx')


gf.print_title('Choose the Data Dictionary')
folder = './DataDictionaries/'
data_dictionary = folder + gf.choose_dir_item(folder,'files', 'csv')

# Excel related: location and which sheet
print('\n'*2)
gf.print_title('Choose the mapping file')
folder = './'
filename = gf.choose_dir_item(folder,'files','xlsx')
mapping_sheet = 'Conversion_Table'

# Read the CSV and create codebook
df_codebook = pd.read_csv(data_dictionary)
codebook_variable = 'Variable / Field Name'


******************************************************************************************
Choose the Data Dictionary
------------------------------------------------------------------------------------------
******************************************************************************************
Choose source by number
------------------------------------------------------------------------------------------
[1]  ISARICCOVID19COREFollowup_DataDictionary_2021-11-19.csv	[2]  ISARICCOVID19COREFollowup_DataDictionary_2021-11-19-Copy1.csv	



Please choose Q(uit) or between 1 and 2:  2





******************************************************************************************
Choose the mapping file
------------------------------------------------------------------------------------------
******************************************************************************************
Choose source by number
------------------------------------------------------------------------------------------
[1]  conversion_remapping_20220105-141343.xlsx	[2]  conversion_remapping.xlsx     	



Please choose Q(uit) or between 1 and 2:  2


In [3]:
print(df_codebook.columns)

# Adding 'id' to codebook for later processing, assign it index value + 1
df_codebook['id'] = df_codebook.index+1
df_codebook['id'] = df_codebook['id'].astype('string')
print(df_codebook['id'])

Index(['Variable / Field Name', 'Form Name', 'Section Header', 'Field Type',
       'Field Label', 'Choices, Calculations, OR Slider Labels', 'Field Note',
       'Text Validation Type OR Show Slider Number', 'Text Validation Min',
       'Text Validation Max', 'Identifier?',
       'Branching Logic (Show field only if...)', 'Required Field?',
       'Custom Alignment', 'Question Number (surveys only)',
       'Matrix Group Name', 'Matrix Ranking?', 'Field Annotation'],
      dtype='object')
0          1
1          2
2          3
3          4
4          5
        ... 
1029    1030
1030    1031
1031    1032
1032    1033
1033    1034
Name: id, Length: 1034, dtype: string


In [4]:
# define the column names of the codebook
columns = ['Variable / Field Name', 'Form Name', 'Field Type','Field Label', 'Choices, Calculations, OR Slider Labels']

# create a dictionary to quickly iterate over a number of columns, the key corresponds with Codebook, the number
# the column # in Excel
dictionary = {'id': 1, 'Variable / Field Name': 3, 'Form Name': 2, 'Field Type': 5,'Field Label': 4, 'Choices, Calculations, OR Slider Labels': 7}

# function for yes/no result based on the answer proded as argument
def answer(reply):
    '''
    To prompt the user to accept the next step, false results in halting the script
    True: C(onctiue), or c(continue)
    False: S(top), s(top), ''
    '''
    yes = set (['C', 'c'])
    no = set (['S', 's', ''])
    
    while True:
        choice = input(reply).lower()
        if choice in yes:
            print('\nScript continues....')
            return True
        elif choice in no:
            print('\nScript has halted, please make changes and rerun the notebook.')
            print('\n\n\n\n')
            return False
        else:
            print("Please respond with 'C' or 'S'\n")


'''
Reading the Mapping Excel and checking with latest Codebook
- When I_variable no longer present in Codebook mark red
- When Codebook has a new variable:
   - insert it in I_variable and relevant items as new row
   - I_variable green
'''
# load appropiate Excel
wb = xl.load_workbook(filename)

# select appropiate sheet
sheet = wb[mapping_sheet]


# convert df_codebook into rows
# rows = dataframe_to_rows(df_codebook)

# create the isaric_mapping_set enabling to check if the Codebook has new variables
isaric_list = []
for col in wb[mapping_sheet]['C']:
    isaric_list.append(col.value)
# isaric_mapping_set = set(isaric_list)

# create the isaric_website_set enabling to check if the mapping has obsolete variabless
# isaric_website_set = set(df_codebook['variable'])

# starting rows for comparing
mapping_row = 2
# codebook_row = 0

# to keep track when the iteration can stop, minus 1 of course
mapping_rows = len(isaric_list) - 1
# codebook_rows = len(df_codebook.index) - 1

# set with Codebook variables
codebook_variables = set(df_codebook[codebook_variable])



# column with variable to check
col_variable = 3

# list of IDs that match
id_list = []
var_list = []

# comparing to existing items in Excel
while mapping_row < mapping_rows:
    # get the variable name of a row in Excel
    check_excel = sheet.cell(row=mapping_row, column=col_variable).value
    # does the excel variable exit in the Codebook
    if check_excel in codebook_variables:
        # get  the right row of the codebook
        codebook_index = df_codebook[df_codebook[codebook_variable] == check_excel].index.to_list()
        # iterate over the Excel row
        for key, col_id in dictionary.items():
            # update the items and mark blue when there is an update.
            if (col_id <= 7) and (sheet.cell(row=mapping_row, column=col_id).value !=str(df_codebook.iloc[codebook_index][key].item())):
                sheet.cell(row=mapping_row, column=col_id, value=str(df_codebook.iloc[codebook_index][key].item()))
#                 orange_color = 'ffb300'
                orange_color = 'f0cc78'
                sheet.cell(row=mapping_row, column=col_id).fill = PatternFill(start_color=orange_color, end_color=orange_color, fill_type='solid')
    else:
        # if the excel does no longer exist in Codebook, mark red
#         red_color = 'FF1E1E'
        red_color = 'FF6969' #'ff9696'
        sheet.cell(row=mapping_row, column=col_variable).fill = PatternFill(start_color=red_color, end_color=red_color, fill_type='solid')
        # remove the ID number in column A
        sheet.cell(row=mapping_row, column=1, value = '')
    # list needed for inserting new items, index is needed to identify the rows.
    id_list.append(sheet.cell(row=mapping_row, column=1).value)
    var_list.append(sheet.cell(row=mapping_row, column=3).value)
    mapping_row += 1    

# set counter for keeping track
counter = 0

# fill the missing
# for x, item in enumerate(df_codebook['id']):
for x, item in enumerate(zip(df_codebook['id'], df_codebook[codebook_variable])):
    if item[1] not in var_list:
        # insert a new row in Excel
        insert_row = counter + 2
        sheet.insert_rows(insert_row)
        # fill the columns
        for key, col_id in dictionary.items():
            sheet.cell(row=insert_row, column=col_id, value=str(df_codebook.iloc[x][key]))
        # color the variable green as for new variable
        #green_color = '64ff64'
        green_color = 'B9FFB9' #'b5ffd8'
        sheet.cell(row=insert_row, column=col_variable).fill = PatternFill(start_color=green_color, end_color=green_color, fill_type='solid')
        
        # add the new item to the Excel id_list
        id_list.insert(counter, item[0])
        counter += 1
    else:
        # set the counter correctly, there might be 'old' variables that take up space
        counter = id_list.index(item[0]) + 1



In [5]:
# save the Excel
def save_the_excel(filename):
    try:
        wb.save(filename=filename)
        print('saved: ' + filename)
    except:
        print('====================================================')
        print('Close ' + filename + ' and Continue, or Stop  ')
        print('====================================================')
        pyanswer = answer('Respond with C(ontinue) or S(top) ')
        if not pyanswer:
            sys.exit('You have halted the operation, please rerun this notebook when you are ready.')
        else:
            save_the_excel(filename)

stamp = f'{datetime.now():%Y%m%d-%H%M%S}'
file_name = filename[:-5] + f"_{stamp}.xlsx"
save_the_excel(file_name)


saved: conversion_remapping_20220105-142914.xlsx
